<a href="https://colab.research.google.com/github/dhanavishnu13/paddy_pest_disease_classification/blob/main/paddy_pest_disease_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Deployed in Huggingface Spaces](https://huggingface.co/spaces/vishnu23/Paddy_Leaf_Disease_Classification_using_CNN)

In [ ]:
#connect google drive to load training images
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Step 1:  Import the required libraries and packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import keras
import tensorflow
from keras.preprocessing.image  import ImageDataGenerator
from keras.layers import Flatten, Dense
from keras import layers,models
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
#Step 2: Loading the dataset
'''
I have the paddy pest disease classification dataset in my drive.
ImageDataGenerator can apply various transformations, such as rotation, flipping, scaling, shearing,
and more, to generate new variations of the input images in real-time.
This helps the model to generalize better.
'''
train_datagen = ImageDataGenerator(zoom_range=0.5,shear_range=0.3,rescale=1/255,horizontal_flip=True)
train = train_datagen.flow_from_directory(directory='drive/My Drive/paddy-disease-classification/train_images',target_size=(256,256),batch_size=32)


In [ ]:
'''
Showcasing three images from the dataset
'''
t_img,label = train.next()
def plotImage(imgarr,label):
  for im,l in zip(imgarr,label):
    plt.figure(figsize=(5,5))
    plt.imshow(im)
    plt.show()

plotImage(t_img[:3],label[:3])

In [ ]:
#Step 3: About the CNN model:
'''
There are 12 layers total in this CNN model, 6 of which are conv2D levels and the remaining two are MaxPooling2D layers.
They are alternately placed one on top of the other. The input shape is set at 256x256.
I previously downsized the photos to 256x256, thus I'm using 256x256 for the input shape.

Relu can be replaced with other terms like tanh, sigmoid, leaky relu, and many others.

I'm using softmax at the output layer because our model is a multi-class classification model with a total of 10 classes.

By the end of the 100th period, my accuracy was about 70%.

I picked the Adam optimizer because it is essentially a standard optimizer that we employ.

SGD, AdaGrad, AdaDelta, RMSProp, and other tools are alternatives to Adam.
'''
model = models.Sequential([
    layers.Conv2D(128, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128 (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
])
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train, epochs=100,steps_per_epoch=100)
model.save('model_extended.h5')
model.save('model.keras')

In [ ]:
#Step 4: About Model Prediction:
'''
I saved the model as model in the cell above.h5
I'm importing the model into this cell and preparing it using a few different methods.

the input image that will be used as a parameter in our model to determine its class.

I'm reading the image file since I specified a 256x256 input dimension in the model design.

transforming it into an array before scaling it down to 256x256.

Prior to sending the photographs to the model, I additionally normalise them.

The model result ranges from 0 to 9.

To access the class of the image, I'm using it as the index value.

'''
img = load_img('/content/100042.jpg', target_size=(256, 256))
img_array = img_to_array(img)
img_array = img_array.reshape((1, 256, 256, 3))
img_array = img_array / 255.0
predictions = model.predict(img_array)
predicted_class_index = predictions.argmax()
class_labels = ['bacterial_leaf_blight', 'bacterial_leaf_streak', 'bacterial_panicle_blight','blast','brown_spot','dead_heart','downy_mildew','hispa','normal','tungro' ]  # Replace with your actual class labels
predicted_class_label = class_labels[predicted_class_index]
print(predicted_class_label)